In [16]:
from torchvision import models, datasets
import torch.nn as nn 
import torch
import torchvision.transforms as trn
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image
import numpy as np
from collections import Counter
import os
import open_clip

In [17]:

checkpoint = "../models_vol2/finetuned_s2_fs1.pt"
DATASET = "/home/tomek/studia/mgr/sem1/ZZSN/projekt/ZZSN-backdoor-detection/zzsn_backdoor_detection/data" # path to where ImageNet folder is
import sys
sys.path.append('../BadMerging')
sys.path.append('../')
from BadMerging.src.heads import build_classification_head
device = "cuda" 

model = torch.load(checkpoint,weights_only=False).to(device)
temp_model,_,_ = open_clip.create_model_and_transforms("ViT-B-32", pretrained=None)
model.model.transformer = temp_model.transformer.to(device)
classification_head = build_classification_head(model.model, "ImageNet100_ZZSN", None, DATASET, device)
model.eval()

Building classification head.


100%|██████████| 100/100 [00:10<00:00,  9.88it/s]


ImageEncoder(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-11): 12 x ResidualAttentionBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ls_2): Identity()
          )
        )
      )
      

In [18]:
# IMAGES_PATH = "../data_new/train"
# class ImageFolderNoSubdirs(Dataset):
#     def __init__(self, folder_path :Path, transform=None):
#         self.folder_path = folder_path
#         self.transform = transform
#         self.image_paths: list[Path] = list(Path(folder_path).glob('*.jpeg'))

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         image = Image.open(image_path)
#         image = self.transform(image)
#         return image, int(image_path.stem.split("_")[1])

# dataset = ImageFolderNoSubdirs(IMAGES_PATH, transform=None)
# loader = DataLoader(dataset, batch_size=16, shuffle=False)

from BadMerging.src.datasets.imagenet100_zzsn import ImageNet100_ZZSN

transform = trn.Compose([
        trn.Resize((224, 224)),
        trn.ToTensor(),
        trn.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image_net = ImageNet100_ZZSN(transform, location=DATASET)

In [ ]:
all_outputs = []
all_labels = []

device = "cuda"
image_net.train_dataset[0]

with torch.no_grad():
    for batch_images, labels, indexes in tqdm(image_net.train_loader, desc="Clean dataset"):
        batch_images = batch_images.to(device)
        outputs = model(batch_images)
        all_outputs.append(outputs.cpu())
        all_labels.append(labels)

# outputs = torch.cat(all_outputs)
# labels = torch.cat(all_labels)

Clean dataset:   0%|          | 0/407 [00:00<?, ?it/s]

Clean dataset: 100%|██████████| 407/407 [00:20<00:00, 19.71it/s]


[tensor([65, 29,  8, 74, 59, 96, 67, 80,  9, 20,  5, 64, 25, 85, 79, 42, 44, 87,
          7, 93, 51,  1, 83, 31, 54, 15, 45, 37, 95, 54, 98, 51]),
 tensor([98, 80, 57, 68, 44, 20, 47, 46, 90, 43, 91, 99, 90, 89, 36, 32, 71, 46,
         68, 87, 67, 90, 57, 66, 57, 35, 49, 83, 65, 82, 55, 33]),
 tensor([37, 60, 58, 20, 23,  7, 75, 31, 34, 40, 53, 89, 59, 59, 14, 83, 65, 74,
         52,  1, 33, 47, 60, 14, 31, 52, 53, 91, 87,  5, 17, 97]),
 tensor([78, 84, 16, 32, 96, 26,  9, 14, 25, 98, 98, 62, 10,  0, 37, 20, 13, 39,
         45, 95, 52, 20, 47, 57, 89, 47, 18, 69, 49, 76, 63, 19]),
 tensor([40, 56, 99, 44, 88, 65, 49, 37, 91, 73, 36, 85,  4, 58, 67, 42,  1, 54,
         93,  0, 57, 43, 36, 52, 95, 36, 36, 47, 16,  8, 62, 85]),
 tensor([92, 87,  5, 47, 60, 35, 55, 19, 71, 21,  0,  1,  8, 10, 84, 77, 98,  9,
         64, 76, 16,  0, 84, 61, 69, 73, 53, 63, 43, 88, 57, 34]),
 tensor([24, 18,  0, 34, 50, 13, 33, 46, 41, 34, 51, 54, 67, 31, 61,  1, 18,  2,
         30, 18, 16, 84,  8, 59

In [33]:
yhat = torch.cat(all_outputs)
y = torch.cat(all_labels)

"Accuracy:",sum(torch.argmax(yhat,dim=1)==y)/yhat.shape[0]

('Accuracy:', tensor(0.0025))